In [1]:
%aiida
%load_ext autoreload
%autoreload 2

In [2]:
from twinpy.api_twinpy import get_twinpy_from_cell
from twinpy.lattice.lattice import get_hexagonal_lattice_from_a_c
from twinpy.properties.hexagonal import get_atom_positions
from twinpy.structure.base import get_cell_from_phonopy_structure
from twinpy.file_io import write_poscar
import spglib
import numpy as np

In [3]:
lattice = get_hexagonal_lattice_from_a_c(a=2.93, c=4.65)
scaled_positions = get_atom_positions(wyckoff='c')
symbols = ['Ti'] * 2
cell = (lattice, scaled_positions, symbols)
twinpy = get_twinpy_from_cell(cell, twinmode='10-12')
# twinpy = get_twinpy_from_cell(cell, twinmode='11-21')

In [225]:
ratio = 0.3
twinpy.set_shear(shear_strain_ratio=ratio)

In [226]:
twinpy.write_shear_structure(structure_type='base', filename = '/home/mizo/fuga/base.poscar')
twinpy.write_shear_structure(structure_type='primitive', filename = '/home/mizo/fuga/primitive.poscar')
twinpy.write_shear_structure(structure_type='conventional', filename = '/home/mizo/fuga/conventional.poscar')

/home/mizo/fuga/base.poscar
/home/mizo/fuga/primitive.poscar
/home/mizo/fuga/conventional.poscar


In [227]:
base = twinpy.get_shear_phonopy_structure(structure_type='base')
primitive = twinpy.get_shear_phonopy_structure(structure_type='primitive')
conventional = twinpy.get_shear_phonopy_structure(structure_type='conventional')
base_cell = get_cell_from_phonopy_structure(base)
primitive_cell = get_cell_from_phonopy_structure(primitive)
conventional_cell = get_cell_from_phonopy_structure(conventional)
base_primitive_cell = twinpy.shear.get_base_primitive_cell(ratio=ratio)
write_poscar(filename='/home/mizo/fuga/base_primitive.poscar', *base_primitive_cell)

/home/mizo/fuga/base_primitive.poscar


In [228]:
base_cell

(array([[ 1.465     , -2.53745443,  0.        ],
        [ 4.395     ,  2.53745443,  4.65      ],
        [-4.16500305, -2.40466563,  4.89334148]]),
 array([[ 0.5       ,  0.20833333,  0.04166667],
        [-0.5       , -0.20833333, -0.04166667],
        [ 1.        ,  0.45833333,  0.79166667],
        [ 0.        ,  0.04166667,  0.70833333],
        [ 0.5       ,  0.70833333,  0.54166667],
        [-0.5       ,  0.29166667,  0.45833333],
        [ 1.        ,  0.95833333,  0.29166667],
        [ 0.        ,  0.54166667,  0.20833333]]),
 ['Ti', 'Ti', 'Ti', 'Ti', 'Ti', 'Ti', 'Ti', 'Ti'])

In [229]:
primitive_cell

(array([[ 2.47180878, -1.465     ,  0.        ],
        [ 2.47180878,  1.465     ,  0.        ],
        [-0.01530974,  0.        ,  4.77349349]]),
 array([[0.66666667, 0.66666667, 0.75      ],
        [0.33333333, 0.33333333, 0.25      ]]),
 ['Ti', 'Ti'])

In [230]:
conventional_cell

(array([[ 4.94361755,  0.        ,  0.        ],
        [ 0.        ,  2.93      ,  0.        ],
        [-0.01530974,  0.        ,  4.77349349]]),
 array([[0.16666667, 0.5       , 0.75      ],
        [0.83333333, 0.5       , 0.25      ],
        [0.66666667, 0.        , 0.75      ],
        [0.33333333, 0.        , 0.25      ]]),
 ['Ti', 'Ti', 'Ti', 'Ti'])

In [231]:
base_primitive_cell

(array([[ 2.87250076, -0.0331972 , -0.06083537],
        [-1.465     ,  2.53745443,  0.        ],
        [ 0.11499847,  0.0663944 ,  4.77167074]]),
 array([[0.33333333, 0.66666667, 0.25      ],
        [0.66666667, 0.33333333, 0.75      ]]),
 ['Ti', 'Ti'])

In [232]:
reduced_lattice = spglib.delaunay_reduce(base_primitive_cell[0])
reduced_scaled_positions = np.dot(np.dot(np.linalg.inv(reduced_lattice.T), base_primitive_cell[0].T),
                                  base_primitive_cell[1].T).T % 1
reduced_symbols = ['Ti'] * 2
reduced_cell = (reduced_lattice, reduced_scaled_positions, reduced_symbols)
write_poscar(filename='/home/mizo/fuga/reduced.poscar', *reduced_cell)

/home/mizo/fuga/reduced.poscar


In [233]:
P = np.dot(reduced_cell[0].T, np.linalg.inv(primitive_cell[0].T))

In [234]:
np.dot(P, primitive_cell[0].T)

array([[ 1.40750076,  2.87250076, -0.11499847],
       [ 2.50425723, -0.0331972 , -0.0663944 ],
       [-0.06083537, -0.06083537, -4.77167074]])

In [235]:
reduced_cell[0].T

array([[ 1.40750076,  2.87250076, -0.11499847],
       [ 2.50425723, -0.0331972 , -0.0663944 ],
       [-0.06083537, -0.06083537, -4.77167074]])

In [236]:
a = np.dot(np.dot(np.linalg.inv(P), reduced_cell[0].T), reduced_cell[1].T).T

In [237]:
b = np.dot(primitive_cell[0].T, primitive_cell[1].T).T

In [238]:
c = np.dot(np.linalg.inv(primitive_cell[0].T), a.T).T
c

array([[0.66666667, 0.66666667, 0.75      ],
       [0.33333333, 0.33333333, 0.25      ]])

In [239]:
d = np.dot(np.linalg.inv(primitive_cell[0].T), b.T).T
d

array([[0.66666667, 0.66666667, 0.75      ],
       [0.33333333, 0.33333333, 0.25      ]])

In [240]:
(c - d) % 1

array([[3.33066907e-16, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [241]:
reduced_cell

(array([[ 1.40750076,  2.50425723, -0.06083537],
        [ 2.87250076, -0.0331972 , -0.06083537],
        [-0.11499847, -0.0663944 , -4.77167074]]),
 array([[0.66666667, 0.66666667, 0.75      ],
        [0.33333333, 0.33333333, 0.25      ]]),
 ['Ti', 'Ti'])

In [242]:
d

array([[0.66666667, 0.66666667, 0.75      ],
       [0.33333333, 0.33333333, 0.25      ]])

In [243]:
lat = primitive_cell[0]
posi = a
symbols = ['Ti'] * 2
cell = (lat, posi, symbols)
write_poscar(filename='/home/mizo/fuga/hoge.poscar', *cell)

/home/mizo/fuga/hoge.poscar


In [244]:
np.linalg.inv(P)

array([[ 8.65763073e-01,  4.99848543e-01, -2.46116814e-02],
       [ 5.00000000e-01, -8.66025404e-01,  1.05130167e-17],
       [-2.13143413e-02, -1.23058407e-02, -9.99697087e-01]])

In [245]:
P.T

array([[ 8.65763073e-01,  4.99848543e-01, -2.46116814e-02],
       [ 5.00000000e-01, -8.66025404e-01, -1.54117944e-18],
       [-2.13143413e-02, -1.23058407e-02, -9.99697087e-01]])

In [246]:
base_primitive_cell

(array([[ 2.87250076, -0.0331972 , -0.06083537],
        [-1.465     ,  2.53745443,  0.        ],
        [ 0.11499847,  0.0663944 ,  4.77167074]]),
 array([[0.33333333, 0.66666667, 0.25      ],
        [0.66666667, 0.33333333, 0.75      ]]),
 ['Ti', 'Ti'])

In [247]:
X_r_bar = np.dot(np.dot(np.linalg.inv(primitive_cell[0].T), base_primitive_cell[0].T),
                 base_primitive_cell[1].T).T % 1

In [248]:
(primitive_cell[1] - base_primitive_cell[1]) % 1

array([[3.33333333e-01, 1.00000000e+00, 5.00000000e-01],
       [6.66666667e-01, 1.66533454e-16, 5.00000000e-01]])

In [257]:
primitive_cell[0].T

array([[ 2.47180878,  2.47180878, -0.01530974],
       [-1.465     ,  1.465     ,  0.        ],
       [ 0.        ,  0.        ,  4.77349349]])

In [252]:
base_primitive_cell

(array([[ 2.87250076, -0.0331972 , -0.06083537],
        [-1.465     ,  2.53745443,  0.        ],
        [ 0.11499847,  0.0663944 ,  4.77167074]]),
 array([[0.33333333, 0.66666667, 0.25      ],
        [0.66666667, 0.33333333, 0.75      ]]),
 ['Ti', 'Ti'])

In [254]:
reduced_lattice.T

array([[ 1.40750076,  2.87250076, -0.11499847],
       [ 2.50425723, -0.0331972 , -0.0663944 ],
       [-0.06083537, -0.06083537, -4.77167074]])

In [264]:
# lattice = [[7.17851431, 0, 0],  # a
#            [0, 3.99943947, 0],  # b
#            [0, 0, 8.57154746]]  # c
# points = [[0.0, 0.84688439, 0.1203133],
#           [0.0, 0.65311561, 0.6203133],
#           [0.0, 0.34688439, 0.3796867],
#           [0.0, 0.15311561, 0.8796867],
#           [0.5, 0.34688439, 0.1203133],
#           [0.5, 0.15311561, 0.6203133],
#           [0.5, 0.84688439, 0.3796867],
#           [0.5, 0.65311561, 0.8796867]]
# numbers = [35,] * len(points)
# cell = (lattice, points, numbers)
cell = list(base_cell)
cell[2] = [22] * 8
cell = tuple(cell)
dataset = spglib.get_symmetry_dataset(cell)
print("Space group type: %s (%d)"
      % (dataset['international'], dataset['number']))
print("Transformation matrix:")
for x in dataset['transformation_matrix']:
    print("  %2d %2d %2d" % tuple(x))
print("Origin shift: %f %f %f" % tuple(dataset['origin_shift']))

Space group type: C2/m (12)
Transformation matrix:
   0  0  0
   0  0  0
   0 -1  0
Origin shift: 0.000000 0.000000 0.000000


(array([[ 2.87250076, -0.0331972 , -0.06083537],
        [-1.465     ,  2.53745443,  0.        ],
        [ 0.11499847,  0.0663944 ,  4.77167074]]),
 array([[0.33333333, 0.66666667, 0.25      ],
        [0.66666667, 0.33333333, 0.75      ]]),
 ['Ti', 'Ti'])